In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.feature import hog


def intersection_over_union(coord, coord_label):
    """
    coord : (x_haut_gauche, y_haut_gauche, x_bas_droite, y_bas_droite)
    coord_label : (x_haut_gauche, y_haut_gauche, x_bas_droite, y_bas_droite)
    """

    x1, y1, x2, y2 = coord
    x1_l, y1_l, x2_l, y2_l = coord_label

    x_inter_1 = max(x1, x1_l)
    y_inter_1 = max(y1, y1_l)
    x_inter_2 = min(x2, x2_l)
    y_inter_2 = min(y2, y2_l)

    interArea = max(0, x_inter_2 - x_inter_1 + 1) * max(0, y_inter_2 - y_inter_1 + 1)

    boxArea = (x2 - x1 + 1) * (y2 - y1 + 1)
    boxArea_label = (x2_l - x1_l + 1) * (y2_l - y1_l + 1)

    return interArea / (boxArea + boxArea_label - interArea)

In [5]:
train_img = os.path.join("dataset", "train", "images")
train_lab = os.path.join("dataset", "train", "labels")
train_list = os.listdir(train_img)

val_img = os.path.join("dataset", "val", "images")
val_lab = os.path.join("dataset", "val", "labels")
val_list = os.listdir(val_img)

test_img = os.path.join("dataset", "test")
test_list = os.listdir(test_img)

path_new_dataset = os.path.join("full_dataset")
    
subfolders = ["frouge", "fvert", "forange", "interdiction", "danger", "stop", "ceder", "obligation", "none"]

folder_corr = {
    "frouge": 0,
    "fvert": 0,
    "forange": 0,
    "interdiction": 0,
    "danger": 0,
    "stop": 0,
    "ceder": 0,
    "obligation": 0,
    "none": 0
}

for subfolder in subfolders:
    folder_corr[subfolder] = os.path.join(path_new_dataset, subfolder)

## Entrainement

In [6]:
# On ajoute toutes les images des dossiers dans un dataframe
X_train_detector = []
Y_train_detector = []

X_train_classifier = []
Y_train_classifier = []

correspondance = {
    "none": 0,
    "frouge": 1,
    "fvert": 2,
    "forange": 3,
    "interdiction": 4,
    "danger": 5,
    "stop": 6,
    "ceder": 7,
    "obligation": 8
}

taille = 32

for subfolder in subfolders:
    for img in tqdm(os.listdir(os.path.join(path_new_dataset, subfolder))):
        img_data = cv2.imread(os.path.join(path_new_dataset, subfolder, img))
        img_data = cv2.resize(img_data, (taille, taille), interpolation=cv2.INTER_AREA)
        hog_img = hog(cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY), orientations=9, pixels_per_cell=(2, 2), visualize=False)

        X_train_detector.insert(0, hog_img)
        Y_train_detector.insert(0, 0 if subfolder == "none" else 1)
        
        hist_red = cv2.calcHist([img_data], [0], None, [256], [0, 256])
        hist_red = hist_red / hist_red.sum()
        hist_green = cv2.calcHist([img_data], [1], None, [256], [0, 256])
        hist_green = hist_green / hist_green.sum()
        hist_blue = cv2.calcHist([img_data], [2], None, [256], [0, 256])
        hist_blue = hist_blue / hist_blue.sum()
        
        X_train_classifier.insert(0, np.concatenate((hog_img, hist_red.flatten(), hist_green.flatten(), hist_blue.flatten())))
        Y_train_classifier.insert(0, correspondance[subfolder])
        
X_train_detector = np.array(X_train_detector)
Y_train_detector = np.array(Y_train_detector)

X_train_classifier = np.array(X_train_classifier)
Y_train_classifier = np.array(Y_train_classifier) 

100%|██████████| 3792/3792 [00:35<00:00, 106.48it/s]


In [7]:
# Compare with random forest
from sklearn.ensemble import RandomForestClassifier

model_detector = RandomForestClassifier(n_jobs=-1, class_weight="balanced")
model_detector.fit(X_train_detector, Y_train_detector)

model_classifier = RandomForestClassifier(n_jobs=-1, class_weight="balanced")
model_classifier.fit(X_train_classifier, Y_train_classifier)

RandomForestClassifier(class_weight='balanced', n_jobs=-1)

In [ ]:
import joblib

filename_detector = 'detector_32_32_hog.sav'
joblib.dump(model_detector, filename_detector)

filename_classifier = 'classifier_32_32_hog_hist.sav'
joblib.dump(model_classifier, filename_classifier)

## Récupération des modèles et prédiction

In [ ]:
import joblib

# Load the saved models
filename_detector = 'detector_32_32_hog.sav'
model_detector = joblib.load(filename_detector)

filename_classifier = 'classifier_32_32_hog_hist.sav'
model_classifier = joblib.load(filename_classifier)

In [12]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])
            
            
def pyramid_image(image, model_binaire, list=[(1, 1), (2, 2), (4, 4), (8,8), (1/2, 1), (1, 2), (2, 4), (4, 8)]):
    all_window = []
    all_boxes = []

    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    w = image.shape[1]
    h = image.shape[0]
    for scale in tqdm(list):
        img = cv2.resize(image, (int(w/scale[0]), int(h/scale[1])))
        pas = int(48//scale[0])
        for x, y, data in sliding_window(img, stepSize=pas, windowSize=(taille, taille)):
            true_x = int(x*scale[0])
            true_y = int(y*scale[1])
            true_x2 = int(x*scale[0]+taille*scale[0])
            true_y2 = int(y*scale[1]+taille*scale[1])
            all_boxes.insert(0, [true_x, true_y, true_x2, true_y2])
            data_resized = cv2.resize(data, (taille, taille), interpolation=cv2.INTER_LANCZOS4) if data.shape < (taille, taille) else cv2.resize(data, (taille, taille), interpolation=cv2.INTER_AREA)
            hog_data = hog(data_resized, orientations=9, pixels_per_cell=(2, 2), visualize=False)
            all_window.insert(0, hog_data)
            
    # Predict
    all_window = np.array(all_window)
    predictions = model_binaire.predict(all_window)
    boxes = [[box[0], box[1], box[2], box[3]] for box, pred in zip(all_boxes, predictions) if pred != 0]
    return boxes

def non_max_suppression(boxes, seuil_iou=0.5, seuil_score=0.5):
    """
    boxes : list of boxes (x1, y1, x2, y2, score, pred)
    seuil : intersection over union threshold
    """    
    final_boxes = []
    for box in boxes:
        discard = False
        for other_box in boxes:
            if intersection_over_union(box[:4], other_box[:4]) > seuil_iou and box[4] < other_box[4]:
                discard = True
                break
            
        if not discard:
            final_boxes.insert(0, box)
            
    final_boxes = sorted(final_boxes, key=lambda x: x[4], reverse=True)
    final_boxes = [box for box in final_boxes if box[4] >= seuil_score]
    
    return final_boxes

In [15]:
taille = 32
full_list = {}

for i in tqdm(range(len(test_list))):
    img = os.path.join(test_img, test_list[i])
    img_data = cv2.imread(img)
    
    boxes = pyramid_image(img, model_detector)
    
    # On predict pour chaque box
    all_window = []
    position_x_y = []
    
    img_data = cv2.imread(img)
    for box in boxes:
        x1, y1, x2, y2 = map(int, box[:4])
        data = img_data[y1:y2, x1:x2]
        if data.shape < (taille, taille, 3):
            data = cv2.resize(data, (taille, taille), interpolation=cv2.INTER_LANCZOS4)
        elif data.shape > (taille, taille, 3):
            data = cv2.resize(data, (taille, taille), interpolation=cv2.INTER_AREA)
        
        # Calcul HOG
        hog_img = hog(cv2.cvtColor(data, cv2.COLOR_BGR2GRAY), orientations=9, pixels_per_cell=(2, 2), visualize=False)

        # Calcul histogramme de chaque couleur
        hist_red = cv2.calcHist([data], [0], None, [256], [0, 256])
        hist_red = hist_red / hist_red.sum()
        hist_green = cv2.calcHist([data], [1], None, [256], [0, 256])
        hist_green = hist_green / hist_green.sum()
        hist_blue = cv2.calcHist([data], [2], None, [256], [0, 256])
        hist_blue = hist_blue / hist_blue.sum()
        
        # Ajout le tout concatener
        all_window.insert(0, np.concatenate((hog_img, hist_red.flatten(), hist_green.flatten(), hist_blue.flatten())))
        position_x_y.insert(0, (x1, y1, x2, y2))

    # Predict
    all_window = np.array(all_window)
    
    probabilities = model_classifier.predict_proba(all_window)
    predictions  = model_classifier.predict(all_window)
    
    confidence_scores = [probabilities[i][pred] for i, pred in enumerate(predictions)]

    nms_before = [[x1, y1, x2, y2, score, pred] for (x1, y1, x2, y2), score, pred in zip(position_x_y, confidence_scores, predictions) if pred != 0]    
    nms_after = non_max_suppression(nms_before, seuil_iou=0.4, seuil_score=0.20)
    
    # On ecrit dans le fichier de prédiction
    full_list[test_list[i]] = nms_after

100%|██████████| 89/89 [18:59<00:00, 12.80s/it]


In [16]:
# On écrit les prédictions dans les fichiers
v = 0
os.makedirs("predictions", exist_ok=True)
folder = f"predictions//sw_v{v}_test"
os.makedirs(f"{folder}", exist_ok=True)   

for name, v in full_list.items(): 
    with open(f"{folder}//{name.replace('.jpg', '.txt')}", "w") as f:
        for box in v:
            x1, y1, x2, y2, score, pred = box
            f.write(f"{x1} {y1} {x2} {y2} {score} {pred}\n")